# **INF8460 A20 Project: Open-domain questions answering**

<br>

Equipe 8:


*   Cedric Sadeu (Glove, ranking with classification)
*   Mamoudou Sacko (pretraitement + TF-IDF, cosine ranking)
*   Oumayma Messoussi (PCP Bert, ML/DL for ranking)

<br>

---

<br>

In [1]:
!pip install sent2vec transformers pytorch-pretrained-bert


     |████████████████████████████████| 1.4MB 12.3MB/s 
     |████████████████████████████████| 133kB 50.4MB/s 
     |████████████████████████████████| 890kB 42.7MB/s 
     |████████████████████████████████| 2.9MB 24.7MB/s 
     |████████████████████████████████| 133kB 46.9MB/s 
     |████████████████████████████████| 7.0MB 49.6MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=af138a5b0caa14ddc1c7d5fcc7f87576f3c9e9c803973ed53bb7853dae684dfd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.28 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


---

<br>

## **1. Map Google**


In [2]:

from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Colab Notebooks/INF8460/Project/'

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/')


Mounted at /content/drive
bert.py		      data		inf8460_projet_A20_equipe8_clean.ipynb
correction_script.py  data_handling.py	output
create_embeddings.py  hub.py		__pycache__


---

<br>

## **2. Import**

In [3]:
import torch
import hub

import importlib
importlib.reload(hub)
from hub import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


---

<br>

## **3. Load Data**


In [4]:
corpus_data = load_data("corpus", 0)
vals_data = load_data("validation", force_refresh = 0)

corpus_ids = corpus_data[0]
vals_ids = corpus_data[0]

corpus_paragraphs = corpus_data[1]
vals_questions = vals_data[1]

In [5]:
vals_ids = vals_ids[:25]
vals_questions = vals_questions[:25]

---

<br>

## **4. Prepoc Data**

In [6]:
corpus_text = prepoc_data(corpus_paragraphs)
vals_text = prepoc_data(vals_questions)

---

<br>

## **5. Representation**


In [7]:
representation_name = "tfidf"
corpus_emmbeding , vals_emmbeding = get_emmbending(corpus_text, vals_text, representation_name)


---

<br>

## **6. Ordonnancement**

In [8]:
ranking_list = get_ranking_list(corpus_ids, corpus_emmbeding, vals_ids, vals_emmbeding, metrique=cosine, top=25)

[36258, 17123, 17153, 15269, 17133, 19851, 11102, 17109, 54677, 43464, 74862, 17154, 17117, 17134, 50400, 17129, 23144, 17159, 17137, 4540, 2905, 2876, 58498, 63237, 17157] [0.5786004982319862, 0.5848148761574764, 0.6207749461156968, 0.6367113125421824, 0.6458932356148468, 0.659325189043494, 0.661138523572292, 0.6627210066896976, 0.6683261134963289, 0.6739972966527261, 0.6739972966527261, 0.6755048227762445, 0.683245175801729, 0.686315901464075, 0.6936742355099499, 0.6987851059841244, 0.7091191601846072, 0.7093201654224743, 0.7101746122241221, 0.7153157818329048, 0.7154122954625302, 0.7205346621504115, 0.7297871378717817, 0.7297871378717817, 0.7336066993324062]
[61031, 16509, 20940, 81487, 23375, 73437, 67913, 11517, 39439, 58524, 28415, 51684, 8796, 41681, 1654, 64437, 66824, 56606, 50948, 14281, 58210, 21274, 22328, 22064, 44145] [0.6373400584395233, 0.6701586653589657, 0.6865831688267899, 0.6865831688267899, 0.7027715136486539, 0.7116107689678894, 0.715232241000868, 0.71998019841432

In [ ]:
import io
import os
import pandas as pd


output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output"
TOP_N = 20
s = ";"
id = []
top_n = []
top_10 = []

for id_question in ranking_list:
  id.append(id_question) 
  top_n.append( s.join(str(v) for v in ranking_list[id_question][:TOP_N]) )
  top_10.append( s.join(str(v) for v in ranking_list[id_question][:10]) )

passage_submission = {'id': id, 'top_n': top_n, "top_10" : top_10 }
df = pd.DataFrame(passage_submission, columns= list(passage_submission.keys()))
display(df)
df.to_csv (os.path.join(output_path, "passage_submission.csv"), index = False, header=True)



---

<br>

## **7. Answers**

In [9]:
def get_answers(nlp, paragraphs_id, paragraphs, questions_id, questions, ranking_list):

    questions_answers = {}

    for question_id in ranking_list:
        question = questions[question_id]
        top_paragraphs_ids = ranking_list[question_id]
        top_paragraphs_text = getParagraph(top_paragraphs_ids, paragraphs_id, paragraphs)

        questions_answers[question] = []
        for j, paragraph in enumerate(top_paragraphs_text):
            QA_input = { 'question': question, 'context': paragraph }
            answers = nlp(QA_input)
            questions_answers[question].append( answers )

        questions_answers[question] = sorted(questions_answers[question], key = lambda i: i['score'], reverse=True)

    print( question )
    print( questions_answers[question] )

    return questions_answers

In [12]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/electra-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

answers = get_answers(nlp, corpus_ids, corpus_paragraphs, vals_ids, vals_questions, ranking_list)


Where did the tradition of seeing Virgil as a great magician last the longest?
[{'score': 0.6016614437103271, 'start': 1593, 'end': 1604, 'answer': 'Middle Ages'}, {'score': 0.24999523162841797, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 0.2499922513961792, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 0.24998676776885986, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 0.2499845027923584, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 0.2499713897705078, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 0.2495337873697281, 'start': 1, 'end': 2, 'answer': 'P'}, {'score': 1.0988862442218306e-07, 'start': 60, 'end': 69, 'answer': 'Purgatory'}, {'score': 5.072210207401895e-09, 'start': 304, 'end': 360, 'answer': 'Naples but eventually spreading widely throughout Europe'}, {'score': 2.774139529204689e-10, 'start': 7, 'end': 18, 'answer': 'Middle Ages'}, {'score': 2.6528138019621395e-10, 'start': 12, 'end': 32, 'answer': 'Western Roman empire'}, {'score': 3.699663839173972e-11, 'star

---

<br>

## **8. Analyse**

In [16]:
import pandas as pd
df = pd.DataFrame(answers, columns= list(answers.keys()))
display(df)
output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output"
df.to_csv (os.path.join(output_path, "questions_answers.csv"), index = False, header=True)

,What is the oldest piece of Qutb Shahi architecture in Hyderabad?,What was incorporated to the united states in the first half of the 18th century?,What is a popular grain source for brewing beer in Mexico?,On what digital platform were contestants performances released to after airing on American Idol?,Which Democratic candidate was the first to win 40% of the vote in Alaska since 1946?,Who were the two most powerful nations in the 1860's?,Kanye has stated that members from which rap group influenced his early style?,What is attested as a self-identifier in Iranian languages?,What did this menace undermine?,What sub-group of literature emerged in the 21st century?,Who are among the most senior members of the Catholic Church?,What dynasty ruled Armenia in the 10th century?,How many Jews lived in the Bronx in 2011?,What can the Western Han dynasty also be called?,Where was the smallest camp established?,During the sparging process what do startches turn into?,"The first president, George Washington, took office in what year?","In Whitehead's cosmology, what are the only things that fundamentally exist?",What did several British politicians suggest doing in the 19th century?,What is one design element of Gothic construction that is always terminated by small pinnacles?,What does a behaviorial component focus on when defining sexual orientation?,What's the estimated population of Southampton?,What are castros?,What is the style of the saucer domes in the Belgian Suite?,Where did the tradition of seeing Virgil as a great magician last the longest?
0,"{'score': 0.9856228232383728, 'start': 196, 'e...","{'score': 0.24999725818634033, 'start': 1, 'en...","{'score': 0.9999427795410156, 'start': 582, 'e...","{'score': 0.9999313354492188, 'start': 355, 'e...","{'score': 0.24999141693115234, 'start': 1, 'en...","{'score': 0.24999761581420898, 'start': 1, 'en...","{'score': 0.9996843338012695, 'start': 282, 'e...","{'score': 0.9999485015869141, 'start': 511, 'e...","{'score': 0.9999237060546875, 'start': 204, 'e...","{'score': 0.24999630451202393, 'start': 1, 'en...","{'score': 0.9997043609619141, 'start': 150, 'e...","{'score': 0.2499943971633911, 'start': 1, 'end...","{'score': 0.9999618530273438, 'start': 614, 'e...","{'score': 0.9999885559082031, 'start': 673, 'e...","{'score': 0.9582260251045227, 'start': 400, 'e...","{'score': 0.24998772144317627, 'start': 1, 'en...","{'score': 0.9998245239257812, 'start': 413, 'e...","{'score': 0.697924792766571, 'start': 518, 'en...","{'score': 0.6343523859977722, 'start': 119, 'e...","{'score': 0.24999749660491943, 'start': 1, 'en...","{'score': 0.9755465388298035, 'start': 164, 'e...","{'score': 0.999974250793457, 'start': 540, 'en...","{'score': 0.9999799728393555, 'start': 216, 'e...","{'score': 0.9993364810943604, 'start': 434, 'e...","{'score': 0.6016614437103271, 'start': 1593, '..."
1,"{'score': 0.49918004870414734, 'start': 63, 'e...","{'score': 0.24999654293060303, 'start': 1, 'en...","{'score': 0.99993896484375, 'start': 174, 'end...","{'score': 0.9994984865188599, 'start': 18, 'en...","{'score': 0.24999141693115234, 'start': 1, 'en...","{'score': 0.24999737739562988, 'start': 1, 'en...","{'score': 0.9964148998260498, 'start': 83, 'en...","{'score': 0.8555660247802734, 'start': 226, 'e...","{'score': 0.24999725818634033, 'start': 1, 'en...","{'score': 0.24998772144317627, 'start': 1, 'en...","{'score': 0.57749342918396, 'start': 372, 'end...","{'score': 0.2499927282333374, 'start': 1, 'end...","{'score': 0.2499842643737793, 'start': 1, 'end...","{'score': 0.9997254014015198, 'start': 180, 'e...","{'score': 0.24999618530273438, 'start': 1, 'en...","{'score': 0.24998772144317627, 'start': 1, 'en...","{'score': 0.9923986792564392, 'start': 40, 'en...","{'score': 0.2499854564666748, 'start': 1, 'end...","{'score': 0.24999845027923584, 'start': 1, 'en...","{'score': 0.24999284744262695, 'start': 1, 'en...","{'score': 0.8968991041183472, 'start': 311, 'e...","{'score': 0.9983329176902771, 's